# Gene Ontology Enrichment Analysis for Arabidopsis thaliana

This notebook performs GO term enrichment analysis for a given set of Arabidopsis thaliana proteins (ex2.tsv) against the background GO annotations of all A. thaliana proteins (goa_arabidopsis.tsv).

The analysis uses Fisher's exact test with FDR correction to identify overrepresented GO terms.


## Understanding / plan

We will:

1. Load the protein set (ex2.tsv) and the GO annotation table (goa_arabidopsis.tsv).
2. Clean the GO annotations:
 - remove explicit NOT annotations (qualifier starts with NOT)
 - remove IEA evidence (inferred from electronic annotation)
 - split into 3 aspects: Biological Process (P), Molecular Function (F), Cellular Component (C)
3. For each aspect, compute GO-term frequencies in the background and the protein set, then compute the enrichment score:

$$
E_\pi = \log_2\left(\frac{F_{\pi,\;\text{proteinset}}}{F_{\pi,\;\text{background}}}\right)
$$

4. Interpret the results and discuss a suitable statistical test.


## Imports / setup

Install requirements (if needed):

```bash
pip install pandas numpy matplotlib scipy statsmodels
```


In [1]:
import pandas as pd
import numpy as np
from scipy.stats import fisher_exact
from statsmodels.stats.multitest import multipletests
from IPython.display import Latex, display
import requests


def display_full_table(df, max_rows=None, max_cols=None, colwidth=None):
    with pd.option_context(
            "display.max_rows", max_rows,
            "display.max_columns", max_cols,
            "display.max_colwidth", colwidth,
            "display.width", None,
    ):
        display(df)




## Configuration

**Note:** This notebook expects to be run from the `code/` directory.

All file paths are relative to the notebook location.


In [2]:
# File paths (relative to code/ directory)
protein_set_path = "/data/ex2.tsv"
goa_path = "/data/goa_arabidopsis.tsv"
term_info_path = "/data/quickgo_term_info.tsv"  # Optional: GO term descriptions


## Part (a) – Load input data

- ex2.tsv: protein set (Uniprot IDs)
- goa_arabidopsis.tsv: GO annotation table for all proteins in A. thaliana


In [3]:
# (a) read protein set
protein_set_df = pd.read_csv(protein_set_path, sep="\t", header=None, names=["Uniprot"])
protein_set_df = protein_set_df.dropna()
protein_set = protein_set_df["Uniprot"].astype(str).unique()

print("Protein set size (unique Uniprot IDs):", len(protein_set))
protein_set_df.head()


Protein set size (unique Uniprot IDs): 118


,Uniprot
0,Q9SFG0
1,Q9ZVK6
2,Q84WN3
3,O04249
4,Q56ZZ7


In [4]:
# (a) read GOA table
goa_df = pd.read_csv(goa_path, sep="\t", dtype=str)

print("GOA table shape:", goa_df.shape)
print("Columns:", list(goa_df.columns))
goa_df.head()


GOA table shape: (158988, 5)
Columns: ['Uniprot', 'qualifier', 'go_term', 'aspect', 'evidence_code']


,Uniprot,qualifier,go_term,aspect,evidence_code
0,A0A0A7EPL0,acts_upstream_of_or_within,GO:0016925,P,IDA
1,A0A0A7EPL0,acts_upstream_of_or_within,GO:0051176,P,IGI
2,A0A0A7EPL0,enables,GO:0008270,F,IEA
3,A0A0A7EPL0,enables,GO:0016874,F,IEA
4,A0A0A7EPL0,enables,GO:0019789,F,TAS


## Part (b) – Prepare GO term tables

### (b1) Remove explicit NOT annotations
Rows where qualifier starts with NOT are negative assertions (“this GO term is explicitly not associated”), so we remove them.

### (b2) Keep only non-IEA evidence
We remove rows with evidence code IEA (inferred from electronic annotation).

### (b3) Split by GO aspect
The aspect column uses:
- P → Biological Process
- F → Molecular Function
- C → Cellular Component


In [5]:
goa_work = goa_df.copy()
goa_work["qualifier"] = goa_work["qualifier"].fillna("")

# (b1) drop explicit NOT rows (qualifier like "NOT|located_in")
mask_not = goa_work["qualifier"].str.startswith("NOT")

# (b2) drop IEA evidence rows
mask_iea = goa_work["evidence_code"].fillna("") == "IEA"

print("Rows with NOT qualifier:", int(mask_not.sum()))
print("Rows with IEA evidence:", int(mask_iea.sum()))

goa_filt = goa_work.loc[~(mask_not | mask_iea)].copy()
print("After filtering, shape:", goa_filt.shape)

# (b3) split by aspect
goa_P = goa_filt[goa_filt["aspect"] == "P"].copy()
goa_F = goa_filt[goa_filt["aspect"] == "F"].copy()
goa_C = goa_filt[goa_filt["aspect"] == "C"].copy()

print("Aspect sizes (rows):", {"P": len(goa_P), "F": len(goa_F), "C": len(goa_C)})
print("Aspect sizes (unique proteins):",
      {"P": goa_P['Uniprot'].nunique(), "F": goa_F['Uniprot'].nunique(), "C": goa_C['Uniprot'].nunique()})


Rows with NOT qualifier: 1056
Rows with IEA evidence: 47547
After filtering, shape: (110385, 5)
Aspect sizes (rows): {'P': 42717, 'F': 29234, 'C': 38434}
Aspect sizes (unique proteins): {'P': 17430, 'F': 14748, 'C': 15441}


## Part (c) – Enrichment calculation

For each aspect table:

1. Background frequency 
$$
F_{\pi,\;\text{background}} = \frac{\#\text{proteins annotated with }\pi}{\#\text{proteins in aspect table}}
$$


In [6]:
def background_frequency(goa_aspect: pd.DataFrame,
                         go_term_col: str = "go_term",
                         protein_col: str = "Uniprot") -> tuple[pd.DataFrame, int]:
    proteins_bg = pd.Index(goa_aspect[protein_col].dropna().astype(str).unique())
    n_bg = len(proteins_bg)

    bg_counts = (
        goa_aspect
        .groupby(go_term_col)[protein_col]
        .nunique()
        .rename("n_proteins_background")
    )

    bg_df = bg_counts.to_frame()
    bg_df["F_background"] = bg_df["n_proteins_background"] / n_bg if n_bg else np.nan

    return bg_df, n_bg



2. Protein-set frequency (restricted to proteins that appear in the aspect table)
$$
F_{\pi,\;\text{proteinset}} = \frac{\#\text{protein-set proteins annotated with }\pi}{\#\text{proteins in (protein set ∩ aspect table)}}
$$



In [7]:
def proteinset_frequency(goa_aspect: pd.DataFrame,
                         protein_set,
                         go_term_col: str = "go_term",
                         protein_col: str = "Uniprot") -> tuple[pd.DataFrame, int]:
    proteins_bg = pd.Index(goa_aspect[protein_col].dropna().astype(str).unique())

    protein_set_idx = pd.Index(pd.Series(protein_set).dropna().astype(str).unique())
    proteins_int = protein_set_idx.intersection(proteins_bg)
    n_int = len(proteins_int)

    subset = goa_aspect[goa_aspect[protein_col].isin(proteins_int)]
    ps_counts = (
        subset
        .groupby(go_term_col)[protein_col]
        .nunique()
        .rename("n_proteins_proteinset")
    )

    ps_df = ps_counts.to_frame()
    ps_df["F_proteinset"] = ps_df["n_proteins_proteinset"] / n_int if n_int else np.nan

    return ps_df, n_int


3. Enrichment score
$$
E_\pi = \log_2\left(\frac{F_{\pi,\;\text{proteinset}}}{F_{\pi,\;\text{background}}}\right)
$$

If a GO term has zero frequency in the protein set $$
((F_{\pi,\;\text{proteinset}}=0)),$$ the enrichment becomes $$(-\infty).
$$

In [8]:
def enrichment_log2(bg_df: pd.DataFrame, ps_df: pd.DataFrame) -> pd.DataFrame:
    df = bg_df.join(ps_df, how="left")

    # terms absent in protein set -> set their counts/freq to 0
    df["n_proteins_proteinset"] = df["n_proteins_proteinset"].fillna(0).astype(int)
    df["F_proteinset"] = df["F_proteinset"].fillna(0.0)

    # default: absent terms get -inf
    df["enrichment_log2"] = -np.inf

    # compute log2 only where F_proteinset > 0 (no log2(0), no warning)
    mask = df["F_proteinset"] > 0
    df.loc[mask, "enrichment_log2"] = np.log2(df.loc[mask, "F_proteinset"] / df.loc[mask, "F_background"])

    return df.sort_values("enrichment_log2", ascending=False)


In [9]:
# Apply the 3 functions to the 3 aspect tables (P, F, C)

# --- Biological Process (P)
bg_P, n_bg_P = background_frequency(goa_P)
ps_P, n_int_P = proteinset_frequency(goa_P, protein_set)
enrich_P = enrichment_log2(bg_P, ps_P)

# --- Molecular Function (F)
bg_F, n_bg_F = background_frequency(goa_F)
ps_F, n_int_F = proteinset_frequency(goa_F, protein_set)
enrich_F = enrichment_log2(bg_F, ps_F)

# --- Cellular Component (C)
bg_C, n_bg_C = background_frequency(goa_C)
ps_C, n_int_C = proteinset_frequency(goa_C, protein_set)
enrich_C = enrichment_log2(bg_C, ps_C)



In [10]:
def rank_and_pick(enrich_df, aspect_label, top_n=10, bottom_n=10):
    # 1) remove terms absent in protein subset (avoid -inf, as tutor recommended)
    ranked = enrich_df[enrich_df["enrichment_log2"] != -np.inf].sort_values("enrichment_log2", ascending=False).copy()

    ranked["rank"] = np.arange(1, len(ranked) + 1)

    top = ranked.head(top_n).copy()
    bottom = ranked.tail(bottom_n).copy()

    top["aspect"] = aspect_label
    top["direction"] = "top"
    bottom["aspect"] = aspect_label
    bottom["direction"] = "bottom"

    return top, bottom


top_10_enrich_P, bottom_10_enrich_P = rank_and_pick(enrich_P, "P")
top_10_enrich_F, bottom_10_enrich_F = rank_and_pick(enrich_F, "F")
top_10_enrich_C, bottom_10_enrich_C = rank_and_pick(enrich_C, "C")

table = pd.concat(
    [top_10_enrich_P, bottom_10_enrich_P,
     top_10_enrich_F, bottom_10_enrich_F,
     top_10_enrich_C, bottom_10_enrich_C],
    axis=0
)

## Part (d) – Interpretation of results

### (d1) Meaning of enrichment scores

Because $(E_\pi = \log_2(F_\text{proteinset} / F_\text{background}))$:

- E = 5.0 means the GO term is enriched by $(2^5 = 32 times)$ in the protein set compared to the background.
- E = -5.0 means the GO term is depleted by $(2^5 = 32times)$ (i.e., $(1/32)$ of the background frequency).
- E = 0.0 means the GO term has the same relative frequency in protein set and background.

### Quick numerical summary (this run)

After filtering (NOT + IEA removed), the aspect tables contained:

- P (Biological Process): 17430 background proteins, 74 proteins in the intersection (protein set ∩ P-table), 3718 GO terms
- F (Molecular Function): 14748 background proteins, 87 proteins in the intersection, 2253 GO terms
- C (Cellular Component): 15441 background proteins, 95 proteins in the intersection, 759 GO terms

Note: many GO terms have E = -∞ because they are present in the background but absent from the protein set.


### (d2) Examples of high / low enrichment GO terms (IDs)

- selected top 10 and buttom Go terms  (excluding the -inf) with corresponding rank in their scope of table (P,F or C)


In [11]:
table.head()

,n_proteins_background,F_background,n_proteins_proteinset,F_proteinset,enrichment_log2,rank,aspect,direction
go_term,,,,,,,,
GO:0015780,8,0.000459,8,0.108108,7.879832,1,P,top
GO:0015790,3,0.000172,3,0.040541,7.879832,2,P,top
GO:0042946,1,0.000057,1,0.013514,7.879832,3,P,top
GO:0015714,4,0.000229,4,0.054054,7.879832,4,P,top
GO:0042882,1,0.000057,1,0.013514,7.879832,5,P,top


### (d2) GO term descriptions (examples)

Below are short definitions for a few high and low enrichment terms (one can look them up via QuickGO API).


In [12]:

PFC_TO_LONG = {
    "P": "biological_process",
    "F": "molecular_function",
    "C": "cellular_component",
}
LONG_TO_PFC = {v: k for k, v in PFC_TO_LONG.items()}


def fetch_quickgo_term_info(go_ids, chunk_size=100, timeout=30):
    """
    go_ids: list[str]  -> مثل ["GO:0008150", "GO:0003674", ...]
    خروجی: DataFrame با ستون‌های go_term, name, definition, aspect_long, aspect_PFC
    """
    go_ids = [str(x).strip() for x in go_ids if pd.notna(x) and str(x).strip()]
    go_ids = list(dict.fromkeys(go_ids))  # unique, حفظ ترتیب

    base = "https://www.ebi.ac.uk/QuickGO/services/ontology/go/terms/"
    rows = []

    with requests.Session() as s:
        s.headers.update({"Accept": "application/json"})

        for i in range(0, len(go_ids), chunk_size):
            chunk = go_ids[i:i + chunk_size]
            url = base + ",".join(chunk)

            r = s.get(url, timeout=timeout)
            r.raise_for_status()
            data = r.json()

            for term in data.get("results", []):
                definition = term.get("definition")


                if isinstance(definition, dict):
                    definition_text = definition.get("text")
                else:
                    definition_text = definition

                aspect_long = term.get("aspect")
                rows.append({
                    "go_term": term.get("id"),
                    "name": term.get("name"),
                    "definition": definition_text,
                    "aspect_long": aspect_long,
                })

    return (
        pd.DataFrame(rows)
        .dropna(subset=["go_term"])
        .drop_duplicates(subset=["go_term"])
        .reset_index(drop=True)
    )


go_info = fetch_quickgo_term_info(table.index.tolist())
go_info.to_csv("quickgo_term_info.tsv", sep="\t", index=False)

table_with_desc = table.merge(go_info, on="go_term", how="left")
# table column order
order = ["go_term", "aspect_long", "name", "definition", "rank", "enrichment_log2", "F_proteinset",
         "n_proteins_proteinset", "F_background", "n_proteins_background"]
table_with_desc = table_with_desc[order]
table_with_desc = table_with_desc[~table_with_desc["aspect_long"].isna()]
table_with_desc = table_with_desc[
    ~table_with_desc["definition"].fillna("").str.startswith("OBSOLETE")
]
order = ["go_term", "aspect_long", "name", "rank", "enrichment_log2", "F_proteinset",
         "n_proteins_proteinset", "F_background", "n_proteins_background"]
table_with_desc = table_with_desc[order]
table_with_desc=table_with_desc.reset_index()

In [13]:
table_with_desc

,index,go_term,aspect_long,name,rank,enrichment_log2,F_proteinset,n_proteins_proteinset,F_background,n_proteins_background
0,0,GO:0015780,biological_process,nucleotide-sugar transmembrane transport,1,7.879832,0.108108,8,0.000459,8
1,1,GO:0015790,biological_process,UDP-xylose transmembrane transport,2,7.879832,0.040541,3,0.000172,3
2,2,GO:0042946,biological_process,glucoside transport,3,7.879832,0.013514,1,0.000057,1
3,3,GO:0015714,biological_process,phosphoenolpyruvate transport,4,7.879832,0.054054,4,0.000229,4
4,4,GO:0042882,biological_process,L-arabinose transmembrane transport,5,7.879832,0.013514,1,0.000057,1
5,7,GO:1902334,biological_process,fructose export from vacuole to cytoplasm,8,7.879832,0.027027,2,0.000115,2
6,8,GO:0042593,biological_process,glucose homeostasis,9,7.879832,0.013514,1,0.000057,1
7,9,GO:0035436,biological_process,triose phosphate transmembrane transport,10,7.879832,0.054054,4,0.000229,4
8,10,GO:0009414,biological_process,response to water deprivation,77,1.294869,0.040541,3,0.016523,288
9,11,GO:0010150,biological_process,leaf senescence,78,1.165586,0.013514,1,0.006024,105


### (d3) Findings

To interpret the enrichment results, I **excluded GO terms that were not successfully annotated via QuickGO** (i.e., entries with missing `name`/`definition`). The remaining annotated terms show a very consistent overall theme across all three GO aspects: the protein set is strongly biased toward **membrane transport**, particularly the **transmembrane movement of sugars and nucleotide-sugars**, and it is preferentially associated with **membrane compartments of the endomembrane/secretory system**.

**Biological Process (P):** The most enriched biological processes are dominated by highly specific transport terms, such as *nucleotide-sugar transmembrane transport* (GO:0015780), *UDP-xylose transmembrane transport* (GO:0015790), *glucoside transport* (GO:0042946), *phosphoenolpyruvate transport* (GO:0015714), and *triose phosphate transmembrane transport* (GO:0035436). These terms reach very high enrichment (enrichment_log2 ≈ 7.88), meaning they occur at roughly **~2^7.88 ≈ 235×** higher relative frequency in the protein set than in the background. In contrast, the lowest-ranked (still present) BP terms are broader stress/response processes such as *response to water deprivation* (GO:0009414), *response to salt stress* (GO:0009651), *response to abscisic acid* (GO:0009737), and *defense response* (GO:0006952), indicating that general stress responses are not the primary signature of this protein set compared to the transport-related processes.

**Molecular Function (F):** The strongest enrichment signal is again centered on **transmembrane transporter activity**, especially for sugars and nucleotide-sugars. Top terms include *D-ribose transmembrane transporter activity* (GO:0015591), *monosaccharide transmembrane transporter activity* (GO:0015145), and several nucleotide-sugar transporter activities such as *UDP-galactose* (GO:0005459), *UDP-glucose* (GO:0005460), and *UDP-glucuronate* (GO:0005461) transmembrane transporter activity. These show enrichment_log2 ≈ 7.41 (approximately **~2^7.41 ≈ 170×** enrichment). At the lower end, while some transporter-related functions still appear, a generic functional category like *protein binding* (GO:0005515) is **depleted** (enrichment_log2 ≈ -0.42), suggesting the set is more specialized toward transporter functions rather than broad interaction/binding annotations.

**Cellular Component (C):** The enriched cellular components support the same interpretation: the protein set is preferentially localized to membranes in the endomembrane/secretory pathway and related membrane structures, including the *plastid inner membrane* (GO:0009528), *vesicle membrane* (GO:0012506), *cis-Golgi network* (GO:0005801), *Golgi membrane* (GO:0000139), and *plant-type vacuole membrane* (GO:0009705). Conversely, several broadly represented compartments are depleted, including *cytosol* (GO:0005829; enrichment_log2 ≈ -4.17) and *nucleus* (GO:0005634; enrichment_log2 ≈ -2.53), as well as *chloroplast* (GO:0009507) and *mitochondrion* (GO:0005739), indicating that soluble/cytosolic or nuclear localization is comparatively underrepresented.

Overall, after removing unannotated terms, the enrichment analysis indicates that this protein set is highly enriched for **membrane-associated transporters**, particularly those involved in **sugar and nucleotide-sugar transport**, and is biased toward **Golgi/vesicle/vacuole/plastid membrane-associated** cellular locations rather than cytosolic or nuclear compartments.


### (d4) Statistical test for significance — Fisher’s exact test / hypergeometric test

The enrichment score
$$
E_\pi = \log_2\left(\frac{F_{\pi,\text{proteinset}}}{F_{\pi,\text{background}}}\right)
$$
is a descriptive measure and does **not** provide a p-value.

To test whether a GO term $(\pi)$ is significantly overrepresented in the protein set compared to the background, we can use **Fisher’s exact test** (equivalently, a one-sided **hypergeometric overrepresentation test**) on a 2×2 contingency table.

For a given aspect (P/F/C), define:
- Background universe = all unique proteins annotated in that aspect table.
- Protein set considered = the intersection (protein set ∩ background universe).

For each GO term $(\pi)$, build the 2×2 table:

|                              | In protein set | Not in protein set |
|------------------------------|--------------|------------------|
| Annotated with GO $(\pi)$    | (a)          | (b)              |
| Not annotated with GO $(\pi)$ | (c)          | (d)              |

Then, Fisher’s exact test with `alternative="greater"` returns a p-value for enrichment of $(\pi)$.

Because we test many GO terms, p-values should be corrected for multiple testing (e.g., Benjamini–Hochberg FDR).


In [14]:
def fisher_enrichment_all_terms(goa_aspect: pd.DataFrame,
                                protein_set,
                                go_term_col: str = "go_term",
                                protein_col: str = "Uniprot",
                                alternative: str = "greater") -> pd.DataFrame:
    proteins_bg = pd.Index(goa_aspect[protein_col].dropna().astype(str).unique())
    n_bg = len(proteins_bg)

    protein_set_idx = pd.Index(pd.Series(protein_set).dropna().astype(str).unique())
    proteins_int = protein_set_idx.intersection(proteins_bg)
    n_int = len(proteins_int)

    # For each GO term: which background proteins are annotated?
    term_to_proteins_bg = (
        goa_aspect
        .dropna(subset=[go_term_col, protein_col])
        .assign(**{protein_col: lambda d: d[protein_col].astype(str)})
        .groupby(go_term_col)[protein_col]
        .apply(lambda s: set(s.unique()))
    )

    rows = []
    for term, annotated_bg in term_to_proteins_bg.items():
        annotated_ps = annotated_bg.intersection(set(proteins_int))

        a = len(annotated_ps)
        b = len(annotated_bg) - a
        c = n_int - a
        d = (n_bg - n_int) - b

        # Skip degenerate cases where the universe/intersection is empty
        if n_bg == 0 or n_int == 0:
            pval = np.nan
            odds = np.nan
        else:
            odds, pval = fisher_exact([[a, b], [c, d]], alternative=alternative)

        rows.append({
            "go_term": term,
            "a": a, "b": b, "c": c, "d": d,
            "oddsratio": odds,
            "pvalue": pval,
            "n_background_total": n_bg,
            "n_intersection_total": n_int
        })

    res = pd.DataFrame(rows)

    # Multiple testing correction (Benjamini–Hochberg FDR)
    valid = res["pvalue"].notna()
    if valid.any():
        _, qvals, _, _ = multipletests(res.loc[valid, "pvalue"].values, method="fdr_bh")
        res.loc[valid, "qvalue_fdr_bh"] = qvals
    else:
        res["qvalue_fdr_bh"] = np.nan

    return res.sort_values(["pvalue"], ascending=True).reset_index(drop=True)


fisher_P = fisher_enrichment_all_terms(goa_P, protein_set)
fisher_F = fisher_enrichment_all_terms(goa_F, protein_set)
fisher_C = fisher_enrichment_all_terms(goa_C, protein_set)

alpha = 0.05
sig_P = fisher_P[fisher_P["qvalue_fdr_bh"] <= alpha].head(10)
sig_F = fisher_F[fisher_F["qvalue_fdr_bh"] <= alpha].head(10)
sig_C = fisher_C[fisher_C["qvalue_fdr_bh"] <= alpha].head(10)


In [15]:
sig_P

,go_term,a,b,c,d,oddsratio,pvalue,n_background_total,n_intersection_total,qvalue_fdr_bh
0,GO:0008643,17,1,57,17355,5176.052632,1.166994e-40,17430,74,4.338885e-37
1,GO:0015780,8,0,66,17356,inf,7.144880e-20,17430,74,1.328233e-16
2,GO:0051260,9,6,65,17350,400.384615,1.327638e-18,17430,74,1.645386e-15
3,GO:0072334,7,0,67,17356,inf,1.857989e-17,17430,74,1.727001e-14
4,GO:0015770,7,1,67,17355,1813.208955,1.481390e-16,17430,74,1.101561e-13
5,GO:0015713,6,0,68,17356,inf,4.760823e-15,17430,74,2.950123e-12
6,GO:0015739,5,0,69,17356,inf,1.202280e-12,17430,74,5.587597e-10
7,GO:0015749,5,0,69,17356,inf,1.202280e-12,17430,74,5.587597e-10
8,GO:0015786,4,0,70,17356,inf,2.992991e-10,17430,74,1.011631e-07
9,GO:0015714,4,0,70,17356,inf,2.992991e-10,17430,74,1.011631e-07


In [16]:
sig_F

,go_term,a,b,c,d,oddsratio,pvalue,n_background_total,n_intersection_total,qvalue_fdr_bh
0,GO:0015297,36,10,51,14651,1034.188235,4.709168e-75,14748,87,1.060976e-71
1,GO:0051119,18,1,69,14660,3824.347826,2.160959e-40,14748,87,2.434321e-37
2,GO:0005338,14,0,73,14661,inf,2.060244e-32,14748,87,1.547243e-29
3,GO:0005459,12,0,75,14661,inf,8.060378e-28,14748,87,4.540008e-25
4,GO:0008515,10,1,77,14660,1903.896104,3.275094e-22,14748,87,1.475757e-19
5,GO:0008506,9,1,78,14660,1691.538462,5.625987e-20,14748,87,2.112558e-17
6,GO:0015145,8,0,79,14661,inf,1.054733e-18,14748,87,3.394733e-16
7,GO:0005355,7,0,80,14661,inf,1.943477e-16,14748,87,5.473318e-14
8,GO:0005460,6,0,81,14661,inf,3.537129e-14,14748,87,8.854612e-12
9,GO:0022857,12,71,75,14590,32.878873,5.585707e-14,14748,87,1.258460e-11


In [17]:
sig_C

,go_term,a,b,c,d,oddsratio,pvalue,n_background_total,n_intersection_total,qvalue_fdr_bh
0,GO:0005794,41,800,54,14546,13.805231,4.768959e-27,15441,95,3.619640e-24
1,GO:0005887,17,77,78,15269,43.218948,8.306963e-21,15441,95,3.152493e-18
2,GO:0030173,11,19,84,15327,105.637218,1.306029e-17,15441,95,3.304253e-15
3,GO:0016021,16,134,79,15212,22.991876,7.976023e-16,15441,95,1.513450e-13
4,GO:0030176,6,44,89,15302,23.445352,5.906628e-07,15441,95,8.966261e-05
5,GO:0009705,7,120,88,15226,10.092992,1.307762e-05,15441,95,1.654319e-03
6,GO:0005768,9,286,86,15060,5.510652,8.376085e-05,15441,95,6.931938e-03
7,GO:0005802,9,288,86,15058,5.471657,8.819563e-05,15441,95,6.931938e-03
8,GO:0005773,13,597,82,14749,3.916677,8.930119e-05,15441,95,6.931938e-03
9,GO:0000139,5,69,90,15277,12.300322,9.132989e-05,15441,95,6.931938e-03


### Interpretation of Fisher results and answer to (d4)

Using Fisher’s exact test (one-sided, `alternative="greater"`) on the 2×2 contingency table provides a formal p-value for GO-term overrepresentation in the protein set, using the aspect-specific background universe (all proteins annotated in that aspect) and restricting the protein set to the corresponding intersection (protein set ∩ background). Since we test many GO terms, I interpret significance using Benjamini–Hochberg FDR (reported as `qvalue_fdr_bh`).

**Biological Process (P):** The significant hits are strongly dominated by transport-related processes. Terms such as **GO:0015780** (nucleotide-sugar transmembrane transport) and **GO:0015714** (phosphoenolpyruvate transport) show extremely small p-values and remain significant after FDR correction. Several transport processes yield `oddsratio = inf`, which happens when the GO term is observed in the protein subset but not observed at all in the “not in protein set” part of the background (i.e., \(b=0\)); in that scenario the odds ratio becomes infinite, consistent with a very strong enrichment signal. Overall, the BP Fisher results statistically confirm what the enrichment-score ranking already suggested: the protein set is highly enriched for specific transmembrane transport processes.

**Molecular Function (F):** The strongest statistical signal is again in membrane-transport functions. Highly significant terms include **GO:0015297** (antiporter activity; very large odds ratio and extremely small p/q values) as well as multiple transporter activities (e.g., **GO:0015145** monosaccharide transmembrane transporter activity, **GO:0005459** UDP-galactose transmembrane transporter activity, **GO:0005460** UDP-glucose transmembrane transporter activity, **GO:0008506** sucrose:proton symporter activity, and **GO:0022857** transmembrane transporter activity). Several of these also show `oddsratio = inf` due to \(b=0\), again indicating that within the chosen universe the evidence for enrichment is very strong. Taken together, the MF Fisher results support a clear functional interpretation: the protein set is enriched for **specific transmembrane transporter proteins**, including sugar and nucleotide-sugar transporters and related coupled transport mechanisms.

**Cellular Component (C):** The significant CC terms point toward membrane/endomembrane localization. Notably, **GO:0005794** (Golgi apparatus) and **GO:0000139** (Golgi membrane) are significant, along with membrane-associated categories such as **GO:0005887** (integral component of plasma membrane), **GO:0016021** (integral component of membrane), and vacuolar/vesicular components (e.g., **GO:0009705** plant-type vacuole membrane). This provides statistical support that the enriched proteins are not only transporters by function, but are also preferentially associated with membranes and Golgi/endomembrane compartments. As before, any terms labeled **OBSOLETE** (e.g., GO:0030173, GO:0030176 if they appear) should be excluded from biological interpretation, but the overall localization theme is unchanged.

**Answer to (d4):**
A suitable statistical test to assess whether enrichment of a GO term is statistically significant is **Fisher’s exact test** (equivalently, a one-sided **hypergeometric overrepresentation test**) applied to a 2×2 contingency table of annotated vs. non-annotated prot
